In [2]:
!pip install gradio # Install gradio for web app

In [3]:
!pip install flair # Flair for AIn skill detection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=e6f0499c63ca147930d9c8159d4f11fc2df6d95155e77e1d5a58e636703e8027
  Stored in directory: /root/.cache/pip

In [4]:
import gradio as gr # gradio to build the web app
import requests # to fetch the web pages
from bs4 import BeautifulSoup # this library is to parse html
import pandas as pd
from datetime import datetime # to add the dates
import time # import time for adding delays in requests


In [5]:
#import flair tools for AI skill detection
from flair.models import SequenceTagger
from flair.data import Sentence

import threading # for smooth background tasks

import os # to save the files
import random # for random delayes

# import retry tools to handle web errors
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

Load AI Model

In [6]:
# load flair's skill detection model
# flair_model = SequenceTagger.load("flair/ner-english-ontonotes-fast")
flair_model = SequenceTagger.load("kaliani/flair-ner-skill")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/169M [00:00<?, ?B/s]

2025-09-13 11:31:16,248 SequenceTagger predicts: Dictionary with 7 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, <START>, <STOP>


*Define filter Mappings*

In [7]:
# Map experinc levels to LinkedIn URL codes

experience_level_mapping ={
    "Internship":"f_E=1",
    "Entry Level":"f_E=2",
    "Associate":"f_E=3",
    "Mid-Senior Level":"f_E=4",
}

# Map work type to linkedIn URL codes
work_type_mapping = {
    "On-site":"f_WT=1",
    "Hybrid":"f_WT=2",
    "Remote":"f_WT=3"
}

#Map time filters to LinkedIn URL codes
time_filter_mapping = {
    "Past Day":"f_TPR=r86400",
    "Past Week":"f_TPR=r604800",
    "Past Month":"f_TPR=r2592000"
}

In [8]:
Description = """We are hiring - Python Modeling Developer



About the Company



Our Client is a global technology company focused on providing software solutions for the automotive and mobility industry. The company is headquartered in Pune, India, and operates in over 25 countries with delivery centers in Europe, the USA, Japan, China, and India.


About the Role



This role involves maintaining and developing features for the Adoption Curves Model, requiring strong Python modeling skills and the ability to work with various data management tools.


Responsibilities



Python Modeling (core): Maintain and develop features for Adoption Curves Model.
Understand how to work with and add features to a Python calculation model.
Maintain the underlying calculation script, plotting scripts, GUI, and documentation.
Work with Git for version control and manage the model in GitHub.
Develop dashboards using Streamlit (optional) and have experience with frontend/GUI development.
Familiarity with file and data management (Pandas, JSON).
Plotting via Matplotlib, Plotly, Seaborn.
Data analytics skillsets in Python - PySpark (Good to have).
Basic understanding of prompt engineering and LLMs.
Basic exposure to Databricks.
Database experience (SQL, like tools).
Implement a database approach as part of the continued enhancement plan for the tools.


Qualification



Strong Python development experience with object-oriented programming.
Experience with data modeling/calculation frameworks (e.g., numpy-style APIs, model classes).
Familiarity with Pandas, JSON for data manipulation.
Visualization with Matplotlib, Seaborn, or Plotly.
Ability to interpret qualitative inputs and build features based on stakeholder feedback.
Exposure to Databricks platform and basic PySpark usage.
Experience with SQL and database versioning concepts.
Hands-on experience with Git version control and collaborative codebases (GitHub).
Strong communication and collaboration skills.


Required Skills



Communication skills (verbal, written).
Collaborative attitude.
Creativity.
Abstract Thinking.
Willing to work with unclear requirements.
Data Analysis / Stakeholder Input Gathering.



Preferred Skills



Experience with Datbricks.
Basic understanding of prompt engineering and LLMs.




Interested candidates can apply by sharing their resume at techcareers@invokhr.com or apply via LinkedIn job post."""

Skill Detection Function

In [9]:
# Define a function to find skills in description
def get_skills(text):
  #turn text into flair model format
  sentence = Sentence(text)
  #use AI to detect skills
  flair_model.predict(sentence)
  # return the skills found
  return [entity.text for entity in sentence.get_spans("ner")]

get_skills(Description)


['Python Modeling',
 'Python',
 'Python Modeling',
 'Python',
 'GUI',
 'Git',
 'Pandas',
 'JSON',
 'Python',
 'SQL',
 'Python',
 'SQL',
 'Git',
 'via',
 'LinkedIn job post.']

Scrapper manager class

In [10]:
#create a class to manage scrapping
class ScraperManager:
  #setup when class starts
  def __init__(self):
    #flag to stop scrapping
    self.stop_event = threading.Event()
    #Empty table for job data
    self.current_df = pd.DataFrame()
    # Lock to avoid data mix-ups
    self.lock = threading.Lock()

  #reset for new scrape
  def reset(self):
    #clear stop flag
    self.stop_event.clear()
    #clear job table
    self.cuurent_df = pd.DataFrame()

  #add one job to table
  def add_job(self, job_data):
    #Lock to keep data safe
    with self.lock:
      #make job into tiny table
      new_df = pd.DataFrame([job_data])
      #add job to main table
      self.current_df = pd.concat([self.current_df, new_df], ignore_index=True)

#create manage Instance
scraper_manager = ScraperManager()


Save to CSV function

In [11]:
#Define function to save jobs

def save_csv(df, filename = "jobs"):
  try:
    #make folder for files
    os.makedirs("saved_jobs", exist_ok=True)
    #set default name with time stamp
    if not filename:
      filename = f"jobs_{int(time.time())}"
    #build file path
    full_path = f"saved_jobs/{filename}.csv"
    #save table to csv
    df.to_csv(full_path, index=False)
    #confirm save worked
    return f"Saved to {full_path}"
  except Exception as e:
    #show error if save fails
    return f"Save error: {str(e)}"

**Process Job function**

In [12]:
# define functions to extract job details
def process_job(job, work_type, exp_level, position):
    try:
        # find job title
        title_element = job.find("h3", class_="base-search-card__title")
        # find company name
        company_element = job.find("a", class_="hidden-nested-link")
        # find job location
        loc_element = job.find("span", class_="job-search-card__location")
        # find job link
        link_element = job.find("a", class_="base-card__full-link")

        # check all data exists
        if not all([title_element, company_element, loc_element, link_element]):
            return None

        # clean title text
        title = title_element.text.strip()
        # clean company text
        company = company_element.text.strip()
        # clean location text
        loc = loc_element.text.strip()
        # clean link (remove extra bits)
        link = link_element["href"].split("?")[0]

        # setup web session with retries
        session = requests.Session()
        # set retry rules for errors
        retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 503, 504])
        # apply retries to session
        session.mount('https://', HTTPAdapter(max_retries=retries))

        # default if description fails
        desc = "Description not available"
        # empty skills list
        skills = []

        try:
            # wait 2-5 seconds to avoid bans
            time.sleep(random.uniform(2, 5))

            # fetch the job page
            response = session.get(
                link,
                headers={
                    # random browser to seem like human
                    'User-Agent': random.choice([
                        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
                        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"
                    ]),
                    # set language for consistency
                    'Accept-Language': 'en-US,en;q=0.9'
                },
                # timeout after 10 seconds
                timeout=10
            )

            # parse job page HTML
            job_soup = BeautifulSoup(response.text, "html.parser")

            # list of places to find description
            description_selectors = [
                'div.description__text',
                'div.show-more-less-html__markup',
                'div.core-section-container__content',
                'section.core-section-container'
            ]

            # try each description spot
            for selector in description_selectors:
                desc_element = job_soup.select_one(selector)
                if desc_element:
                    # clean description text
                    desc = desc_element.get_text("\n").strip()
                    # find skills with AI
                    skills = get_skills(desc)
                    break

        except Exception as e:
            # log error if page fails
            print(f"Error processing {link}: {str(e)}")

        # return job details
        return {
            "Position": position,
            "Date": datetime.now().strftime("%Y-%m-%d"),
            "Work Type": work_type,
            "Level": exp_level,
            "Title": title,
            "Company": company,
            "Location": loc,
            "Link": f"[{link}]({link})",
            "Description": desc,
            "Skills": ", ".join(skills[:5]) if skills else "No skills detected"
        }

    except Exception as e:
        # log error if job card fails
        print(f"Error processing job card: {str(e)}")
        return None


**Scrapae Job function**

In [13]:
# define function to scrape job listings
def scrape_jobs(location, position, work_types, exp_levels, time_filter):
    # setup web sessions
    session = requests.Session()
    # set retry rules
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    # apply retries
    session.mount('https://', HTTPAdapter(max_retries=retries))

    # loop through work types
    for work_type in work_types:
        # loop through experience levels
        for exp_level in exp_levels:
            # stop if user says so
            if scraper_manager.stop_event.is_set():
                return

            try:
                # Build LinkedIn search URL
                base_url = (
                    f"https://www.linkedin.com/jobs/search/?keywords={position}&location={location}"
                    f"&{work_type_mapping[work_type]}"
                    f"&{experience_level_mapping[exp_level]}"
                    f"&{time_filter_mapping[time_filter]}"
                    f"&radius=0"  # define radius to extract jobs only for specific cities
                )

                try:
                    # Get search page
                    response = session.get(base_url, timeout=10)
                    # parse HTML
                    soup = BeautifulSoup(response.text, "html.parser")
                    # find total job count
                    total_jobs = int(
                        soup.find("h1", class_="results-context-header__job-count").text.replace(",", "")
                    )
                except:
                    # default to 25 if count fails
                    total_jobs = 25

                # cap total_jobs to 100
                total_jobs = min(100, total_jobs)

                # Loop through pages (25 jobs each)
                for start in range(0, total_jobs, 25):
                    # stop if user says so
                    if scraper_manager.stop_event.is_set():
                        return
                    # wait 2-5 seconds
                    time.sleep(random.uniform(2, 5))

                    # add page number to url
                    url = f"{base_url}&start={start}"
                    try:
                        # get page
                        response = session.get(url, timeout=10)
                        # parse HTML
                        soup = BeautifulSoup(response.text, "html.parser")
                        # find all job cards
                        job_cards = soup.find_all("div", class_="base-card")
                    except Exception as e:
                        # log error if page fails
                        print(f"Failed to scrape page {start}: {str(e)}")
                        continue

                    # mix up job order
                    random.shuffle(job_cards)

                    # process each job
                    for job in job_cards:
                        # stop if user says so
                        if scraper_manager.stop_event.is_set():
                            return

                        # get job details
                        job_data = process_job(job, work_type, exp_level, position)
                        # add job to table
                        if job_data:
                            scraper_manager.add_job(job_data)
                            yield  # update app

            except Exception as e:
                # log big error
                print(f"Scraping error: {str(e)}")


**Run scrappre function**

In [14]:
# define function to start scraping
def run_scrapper(cities, states, positions, work_types, exp_levels, time_filter):
    # clear old data
    scraper_manager.reset()

    # split cities into list
    cities_list = [c.strip() for c in cities.split(',') if c.strip()]

    # split states/countries into list
    states_list = [s.strip() for s in states.split(',') if s.strip()]

    # combine cities and states
    locations = [f"{city}, {state}" for city in cities_list for state in states_list]

    # clean and format positions
    positions_list = [p.strip().replace(' ', '%20') for p in positions.split(',') if p.strip()]

    # define scraping task
    def worker():
        # loop through locations
        for loc in locations:
            # loop through positions
            for pos in positions_list:
                # stop if user says so
                if scraper_manager.stop_event.is_set():
                    return
                # run scrape_jobs
                for _ in scrape_jobs(loc, pos, work_types, exp_levels, time_filter):
                    pass  # scrape_jobs yields after each job is added

    # start task in background
    thread = threading.Thread(target=worker)
    thread.start()

    # update app while running
    while thread.is_alive():
        # wait a bit
        time.sleep(0.5)
        # lock data
        with scraper_manager.lock:
            # show progress and table
            yield 'Scraping in progress...', scraper_manager.current_df

    # show final status
    yield (
        "Scraping complete!" if not scraper_manager.stop_event.is_set() else "Scraping stopped.",
        scraper_manager.current_df
    )


Gradio Interface

In [15]:
# Build the Web app
with gr.Blocks() as app:
    # Add title
    gr.Markdown("""
    <div style="text-align: center; color: #f67d3c; font-size: 2em; font-weight:bold; margin:20px 0; padding: 10px">
      AI-Powered LinkedIn Job Scraper
    </div>
    """)

    # create user input section
    with gr.Row():
        with gr.Column():
            # input for the cities
            cities = gr.Textbox(label="Cities (comma-separated)")
            # Input for States
            states = gr.Textbox(label="States/Countries (comma-separated)")
            # input for jobs
            positions = gr.Textbox(label="Positions (comma-separated)")
            # checkbox for work type
            work_types = gr.CheckboxGroup(list(work_type_mapping.keys()), label="Work Types")
            # checkbox for experience level
            exp_levels = gr.CheckboxGroup(list(experience_level_mapping.keys()), label="Experience Levels")
            # dropdown for time
            time_filter = gr.Dropdown(list(time_filter_mapping.keys()), label="Time Filter")

      # Buttons for start/stop
    with gr.Row():
      start_btn = gr.Button("Start Scraping", variant="primary")
      stop_btn = gr.Button("Stop Scraping", variant="secondary")

    # show status
    status = gr.Textbox(label="Status")

    # Show job table
    results = gr.Dataframe(
        headers=["Position", "Date", "Work Type", "Level", "Title", "Company", "Location", "Link", "Skills"],
        datatype=["str", "str", "str", "str", "str", "str", "str", "str", "str"],
        interactive=False
        )

    # save section
    with gr.Row():
        # Input for file name
        filename = gr.Textbox(label="Filename (Optional)", placeholder="my_jobs")
        # save button
        save_btn = gr.Button("Save to CSV", variant="secondary")

    # show save status
    save_status = gr.Textbox(label="Save Status")

    # Prep start: clear any previous stop signal
    start_btn.click(
        lambda: scraper_manager.stop_event.clear(),
        outputs=None
    )

    # Link start button to scraper (streaming updates)
    start_btn.click(
        run_scrapper,
        inputs=[cities, states, positions, work_types, exp_levels, time_filter],
        outputs=[status, results]
    )

    # link stop button to pause
    stop_btn.click(
        lambda: scraper_manager.stop_event.set(),
        outputs=None
    )

    # link save button to save
    save_btn.click(
        save_csv,
        inputs=[results, filename],
        outputs=save_status
    )

# Run the app
if __name__ == "__main__":
    app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6116cc44e97e933c68.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
